In [ ]:
# install dependencies if not exists
!pip3 install tensorflow==2.3

In [1]:
# import modules
import tensorflow
from tensorflow import keras 
import logger 
import os
print(tensorflow.__version__)

2.3.0


In [3]:
# prepare model

# naive model with fully connected
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(128,128,3)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()         

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1572896   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 1,572,962
Trainable params: 1,572,962
Non-trainable params: 0
_________________________________________________________________


In [4]:
# configs
img_height = 128
img_width  = 128
batch_size = 32

# preprocess training data
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=1/3)
train_set = train_datagen.flow_from_directory(
    './faces',
    seed=10, # TODO: Change to time()
    target_size=(128,128),
    class_mode='categorical', 
    batch_size=32, 
    subset="training"
)

test_set = train_datagen.flow_from_directory(
    './faces',
    seed=10, # TODO: Change to time()
    target_size=(128,128),
    class_mode='categorical', 
    batch_size=32, 
    subset="validation"
)

Found 450 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


In [5]:
# train 
model.fit_generator(
    train_set, # data to train, a format of (X, y)
    steps_per_epoch=len(train_set), # The number of batch iterations before a training epoch is considered finished. Ignore if whole data is load, in our case, we need this in order to iterate over batches; hence make sure that generator can generate at least: steps_per_epoch * epochs batches
    epochs=10, # number of epochs, i.e number of time that iteration of updating weights goes
    validation_data=test_set, # validation
    validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
15/15 [==============================] - 1s 37ms/step - loss: 2.4395 - accuracy: 0.5933 - val_loss: 0.4483 - val_accuracy: 0.7500
Epoch 2/10
15/15 [==============================] - 0s 30ms/step - loss: 0.3223 - accuracy: 0.8267 - val_loss: 0.3307 - val_accuracy: 0.9286
Epoch 3/10
15/15 [==============================] - 0s 31ms/step - loss: 0.3122 - accuracy: 0.8356 - val_loss: 0.3393 - val_accuracy: 0.8170
Epoch 4/10
15/15 [==============================] - 0s 31ms/step - loss: 0.1801 - accuracy: 0.8933 - val_loss: 0.2126 - val_accuracy: 0.9152
Epoch 5/10
15/15 [==============================] - 0s 31ms/step - loss: 0.1282 - accuracy: 0.9178 - val_loss: 0.2151 - val_accuracy: 0.9330
Epoch 6/10
15/15 [==============================] - 0s 31ms/step - loss: 0.1149 - accuracy: 0.9133 - val_loss: 0.1381 - val_accuracy: 0.9509
Epoch 7/10
15/15 [==============================] - 0s 30ms/step - loss: 0.110

In [6]:
# save model
model.save('fc_2cls.h5')

In [32]:
logger.debug(len(model.layers[1].get_weights()[0][2]))

[DEBUG, "/usr/lib/python3.8/runpy.py":194, in _run_module_as_main] 32
